In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
train, test = pd.read_csv("/kaggle/input/playground-series-s3e22/train.csv"), pd.read_csv("/kaggle/input/playground-series-s3e22/test.csv")

In [ ]:
train.describe()

In [ ]:
len(train["hospital_number"].unique())

In [ ]:
train = train.drop(["id", "hospital_number"], axis=1)

In [ ]:
train

Converting categorical columns from object to categorical, then convert it to numerical

In [ ]:
def cat_to_numeric(df):
    for col_name in df.columns:
        if(df[col_name].dtype == 'object'):
            df[col_name]= df[col_name].astype('category')
            df[col_name] = df[col_name].cat.codes
    return df

In [ ]:
train = cat_to_numeric(train)

# Naive approach: Random Forest on the full data

In [ ]:
X_train = train.drop(["outcome"], axis=1)
y_train = train["outcome"]

In [ ]:
clf = RandomForestClassifier(random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print(cross_val_score(estimator = clf,
                      X = X_train, 
                      y = y_train, 
                      scoring="f1_weighted", 
                      cv=5, 
                      verbose=4))

# Create submission

In [ ]:
def create_submission(model, test_df):
    ids = test_df["id"]
    
    test_df = test_df.drop(["id", "hospital_number"], axis=1)
    test_df = cat_to_numeric(test_df)
    
    pred = pd.Series(model.predict(test_df))
 
    pred[pred == 0] = "died"
    pred[pred == 1] = "euthanized"
    pred[pred == 2] = "lived"

    submission_df = pd.DataFrame({
        "id": ids,
        "outcome": pred
    })
    
    submission_df.to_csv("submission.csv", index=False)

In [ ]:
create_submission(clf, test)